<a href="https://colab.research.google.com/github/quillaur/data_learning/blob/main/data_science/computer_vision/object_detection/object_detction_using_image_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

## Dataset creation

I am gonna use the jupyter-bbox-widget librairy to annotate images.

* [Documentation](https://github.com/gereleth/jupyter-bbox-widget)
* [Usage exemple](https://github.com/gereleth/jupyter-bbox-widget/blob/main/examples/introduction.ipynb)

In [ ]:
!pip install jupyter_bbox_widget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from jupyter_bbox_widget import BBoxWidget
import ipywidgets as widgets
import json
import os
import numpy as np

from google.colab import output
output.enable_custom_widget_manager()

# Known issue with BBoxWidget => https://github.com/gereleth/jupyter-bbox-widget/issues/10
import base64
def encode_image(filepath):
    with open(filepath, 'rb') as f:
        image_bytes = f.read()
    encoded = str(base64.b64encode(image_bytes), 'utf-8')
    return "data:image/jpg;base64,"+encoded

In [4]:
files = ["/content/ele_4.jpeg", "/content/ele_5.jpeg"]

annotations = {}
annotations_path = 'annotations.json'

In [ ]:
# a progress bar to show how far we got
w_progress = widgets.IntProgress(value=0, max=len(files), description='Progress')
# the bbox widget
w_bbox = BBoxWidget(
    image = encode_image(files[0]),
    classes=["Elephant"]
)

# combine widgets into a container
w_container = widgets.VBox([
    w_progress,
    w_bbox,
])

In [ ]:
# when Skip button is pressed we move on to the next file
@w_bbox.on_skip
def skip():
  w_progress.value += 1
  if w_progress.value < w_progress.max:
    # open new image in the widget
    image_file = files[w_progress.value]
    w_bbox.image = encode_image(image_file)
    # here we assign an empty list to bboxes but 
    # we could also run a detection model on the file
    # and use its output for creating inital bboxes
    w_bbox.bboxes = [] 

# when Submit button is pressed we save current annotations
# and then move on to the next file
@w_bbox.on_submit
def submit():
    image_file = files[w_progress.value]
    # save annotations for current image
    annotations[image_file] = w_bbox.bboxes
    with open(annotations_path, 'w') as f:
        json.dump(annotations, f, indent=4)
    # move on to the next file
    skip()

In [ ]:
w_container

## Modelisation using image AI

https://github.com/OlafenwaMoses/ImageAI/tree/master/imageai/Detection

In [ ]:
! pip install imageai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from imageai.Detection import ObjectDetection
import requests

In [6]:
url = 'https://github.com/OlafenwaMoses/ImageAI/releases/download/3.0.0-pretrained/tiny-yolov3.pt'
r = requests.get(url, allow_redirects=True)
model_filename = "/content/tiny-yolov3.pt"
# Save downloaded content to disk.
open(model_filename, 'wb').write(r.content)

<ipython-input-6-0ff7d30dbf9c>:5: ResourceWarning: unclosed file <_io.BufferedWriter name='/content/tiny-yolov3.pt'>
  open(model_filename, 'wb').write(r.content)


35457043

In [7]:
detector = ObjectDetection()
detector.setModelTypeAsTinyYOLOv3()
detector.setModelPath(model_filename)
detector.loadModel()

In [ ]:
input_img = "/content/ele_4.jpeg"
out_img = "/content/ele_4_out.jpeg"
detections = detector.detectObjectsFromImage(input_image=input_img, output_image_path=out_img, minimum_percentage_probability=30)

In [ ]:
for eachObject in detections:
    print(eachObject["name"] , " : ", eachObject["percentage_probability"], " : ", eachObject["box_points"] )
    print("--------------------------------")

elephant  :  99.75  :  [48, 8, 231, 179]
--------------------------------


## Performance evaluation

https://pypi.org/project/mean-average-precision/

In [ ]:
!pip install mean_average_precision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
from mean_average_precision import MetricBuilder

In [8]:
with open("/content/annotations.json", "r") as f:
  annotated_marks = json.load(f)

In [23]:
annotated_marks

{'/content/ele_4.jpeg': [{'x': 63,
   'y': 13,
   'width': 184,
   'height': 161,
   'label': 'Elephant'}],
 '/content/ele_5.jpeg': [{'x': 56,
   'y': 10,
   'width': 157,
   'height': 182,
   'label': 'Elephant'}]}

In [34]:
metric_fn = MetricBuilder.build_evaluation_metric("map_2d", num_classes=1)

for f_p in files:

  # The ground truth
  gt = []
  for box in annotated_marks[f_p]:
    t = [box["x"], box["y"], box["width"], box["height"]]
    # The class is necessary an elephant here.
    t.append(0)
    # This list must have 7 elems according to the doc.
    t.extend([0,0])
    gt.append(t)

  # The preds
  detections = detector.detectObjectsFromImage(input_image=f_p, minimum_percentage_probability=30)
  preds = []
  for eachObject in detections:
    p = eachObject["box_points"]
    p.extend([0, eachObject["percentage_probability"]])
    preds.append(p)

  metric_fn.add(np.array(preds), np.array(gt))

In [35]:
print(f"VOC PASCAL mAP in all points: {metric_fn.value(iou_thresholds=0.5)['mAP']}")

VOC PASCAL mAP in all points: 1.0
